### Imports

In [2]:
# Install proper version of torch, as according to: https://pytorch.org/
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [4]:
!pip install -q -e ../.

import pandas as pd
import torch
from looptune import prep_config_combinations, single_run, clean_memory
from transformers import Phi3ForSequenceClassification

### Prepare dataset with two columns: 'text' and 'label'

Examplary data source: https://www.kaggle.com/datasets/ankurzing/sentiment-analysis-for-financial-news


In [5]:
df = pd.read_csv('example_data/SentimentAnalysisforFinancialNews.csv', encoding="ISO-8859-1", header=None)
df.columns = ['label', 'text']
df.head(5)

,label,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


### Prepare run configurations

In [11]:
run_config = {   # -----------------------
                  'model_name': [
                           # 'distilbert/distilbert-base-uncased-finetuned-sst-2-english',
                           #  'michellejieli/emotion_text_classifier',
                           #  'cardiffnlp/twitter-xlm-roberta-base-sentiment',
                           #  'celine98/canine-s-finetuned-sst2',
                           #  'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           #  'michelecafagna26/t5-base-finetuned-sst2-sentiment',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'ProsusAI/finbert',
                           # 'arpanghoshal/EmoRoBERTa',
                           # 'camembert-base'
                           # 'cardiffnlp/twitter-roberta-base-irony',
                           # 'cardiffnlp/twitter-roberta-base-sentiment-latest',
                           # 'ctrl',
                           # 'distilroberta-base',
                           # 'flaubert/flaubert_base_cased',
                           # 'j-hartmann/emotion-english-distilroberta-base',
                           # 'joeddav/distilbert-base-uncased-go-emotions-student',
                           # 'lxyuan/distilbert-base-multilingual-cased-sentiments-student',
                           # 'nlptown/bert-base-multilingual-uncased-sentiment',
                           # 'papluca/xlm-roberta-base-language-detection',
                           # 'roberta-base',
                           # 'xlnet-base-cased',
                           # 'facebook/tart-full-flan-t5-xl',
                           # 'lytang/MiniCheck-Flan-T5-Large',
                          # Need peft on 10GB GPU
                            'microsoft/Phi-3-mini-4k-instruct'
                           # 'microsoft/phi-2',
                           #'meta-llama/Meta-Llama-3-8B',
                           #'lightblue/suzume-llama-3-8B-multilingual',
                           # 'google/gemma-2b',
                           # 'mistralai/Mistral-7B-v0.1',
                           #'mistralai/Mistral-7B-Instruct-v0.1',
                           ## 'tiiuae/falcon-11B' ,
                           ], # Pre-trained model names from the Hugging Face hub used for fine-tuning
                    'custom_loader': Phi3ForSequenceClassification,
                 # --------------------------
                 'split': (0.8, 0.2), # Divides the dataset into training, testing, (and optionally) validation sets. Examples: (0.7,0.3) -> split into train and test proportionally; (70, 30) splits into train,test proportionally.
                 'balanced': (('train',), ('test',)),
                 # --------------------------
                 'training_arguments': {
                     'num_train_epochs': 3, # Number of times the model sees the entire training dataset.
                     'per_device_train_batch_size': 16, # Number of samples processed in each training step (personally, 8/16 work best, 16 is faster, but you may find linear drop in inference speed during fine-tuning).
                     'per_device_eval_batch_size': 16, # Number of samples processed in each evaluation step.
                     # 'gradient_accumulation_steps': 4,
                     'gradient_checkpointing': True,
                     #-----------------------------
                     'save_total_limit': 2,
                     'load_best_model_at_end': True,
                     'save_strategy': 'steps', # Controls when to save model checkpoints ('steps', 'epoch' or 'no').
                     'metric_for_best_model': 'f1-score',
                     #-----------------------------
                     'evaluation_strategy': "steps",
                     'logging_steps': 100,
                     #'max_steps': 20,
                     'fp16': False,
                     # 'use_cpu': False,
                     #-----------------------------
                     'learning_rate': 5e-5,
                     'lr_scheduler_type': "linear",
                     'warmup_ratio': 0.1,
                     'max_grad_norm': 0.3,
                     'weight_decay': 0.001,
                 },
                 #-----------------------------
                     'bnb_config': [
                                 #False,
                                {'bnb_4bit_compute_dtype': torch.bfloat16, 'load_in_4bit': True, 'bnb_4bit_quant_type': "nf4", 'bnb_4bit_use_double_quant': True, 'load_in_8bit': False}
                                 ],
                 'peft_config': [
                                #False,
                                {'r': 8, 'lora_alpha': 32, 'lora_dropout': 0.01, 'bias': "none",
                                'task_type': "SEQ_CLS", 
                                 'target_modules': ("v_proj",),
                                'target_modules': "all-linear"
                                }
                                ],
                    }

run_params_serie = prep_config_combinations(run_config)
print(len(run_params_serie))
run_params_serie


1


[{'model_name': 'microsoft/Phi-3-mini-4k-instruct',
  'custom_loader': transformers.models.phi3.modeling_phi3.Phi3ForSequenceClassification,
  'split': (0.8, 0.2),
  'balanced': (('train',), ('test',)),
  'training_arguments': {'num_train_epochs': 3,
   'per_device_train_batch_size': 16,
   'per_device_eval_batch_size': 16,
   'gradient_checkpointing': True,
   'save_total_limit': 2,
   'load_best_model_at_end': True,
   'save_strategy': 'steps',
   'metric_for_best_model': 'f1-score',
   'evaluation_strategy': 'steps',
   'logging_steps': 100,
   'fp16': False,
   'learning_rate': 5e-05,
   'lr_scheduler_type': 'linear',
   'warmup_ratio': 0.1,
   'max_grad_norm': 0.3,
   'weight_decay': 0.001},
  'bnb_config': {'bnb_4bit_compute_dtype': torch.bfloat16,
   'load_in_4bit': True,
   'bnb_4bit_quant_type': 'nf4',
   'bnb_4bit_use_double_quant': True,
   'load_in_8bit': False},
  'peft_config': {'r': 8,
   'lora_alpha': 32,
   'lora_dropout': 0.01,
   'bias': 'none',
   'task_type': 'SEQ_

### Run in loop

In [12]:
for run_params in run_params_serie:
    single_run(run_params, df)

          text
label         
negative   604
neutral   2879
positive  1363


Casting to class labels:   0%|          | 0/4846 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1449 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/363 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

Some weights of Phi3ForSequenceClassification were not initialized from the model checkpoint at microsoft/Phi-3-mini-4k-instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 12,616,728 || all params: 3,735,229,488 || trainable%: 0.3378
GPU = NVIDIA GeForce RTX 3080. Max memory = 10.0 GB.
2.545 GB of memory reserved.


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,1.123000,0.376117,0.836935,0.834711,0.833859,0.834711
200,0.284300,0.521789,0.842538,0.840220,0.834828,0.840220


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


617.1247 seconds used for training.
10.29 minutes used for training.
Peak reserved memory = 5.334 GB.
Peak reserved memory for training = 2.789 GB.
Peak reserved memory % of max memory = 53.34 %.
Peak reserved memory for training % of max memory = 27.89 %.


In [8]:
clean_memory()